In [1]:
import numpy as np
import math

In [74]:
# Finds the index where a linear system is violated

def violation(A,b,x):
    check = (A@x >= b)
    
    for box in range(len(check)):
        if check[box][0] == False:
            return box

In [ ]:
def recenter(center, D, a):
    return 0

In [117]:
# A is a matrix and b is a vector

def Ellipsoid(A,b):
    U = max(A.max(),b.max())
    n = float(len(b))
    
    small_v = np.power(n,-n) * np.power((n*U), (-n**2 * (n+1)))
    big_V = np.power(2*n,n) * np.power(n*U, n**2)
    
    time_limit = math.ceil(2*(n+1) * math.log(big_V/small_v, 2))
    radius = big_V/math.pi
    D = radius * np.identity(int(n))
    center = np.zeros((int(n),1))
    time = 0
    
    while time != time_limit:
        test = (A@center >= b)
        
        if np.all(test):
            return print(center, "|", time)
        
        else:
            index = violation(A,b,center)
            a_horz = np.array(A[index])
            a_vert = a_horz.reshape(-1,1)
            
            center = center + (1/(n+1))*((D@a_vert)/math.sqrt(a_horz@(D@a_vert)))
            #D = (n**2/(n**2 - 1)) * (D - (2/(n+1))*(((D@a_vert)@(a_horz@D))/(a_horz@(D@a_vert))) )
        
        time = time+1
    
    
    return print('P is empty')

In [118]:
A = np.array([[1,0],
              [0,2]])

b = np.array([[3],
              [2]])

Ellipsoid(A,b)

[[27.08110001]
 [27.08110001]] | 2


C:\Users\Ben\AppData\Local\Temp\ipykernel_12300\324504952.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  center = center + (1/(n+1))*((D@a_vert)/math.sqrt(a_horz@(D@a_vert)))
